$\qquad$ $\qquad$$\qquad$  **TDA 231 Machine Learning: Homework 2** <br />
$\qquad$ $\qquad$$\qquad$ **Goal: Classification**<br />
$\qquad$ $\qquad$$\qquad$                   **Grader: Divya** <br />
$\qquad$ $\qquad$$\qquad$                     **Due Date: 23/4** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Sandra Viknander, 9003012482, danvik@student.chalmers.se ; Kevin Jaquier, 921119T334, jaquier@student.chalmer.se** <br />

General guidelines:
* All solutions to theoretical problems, can be submitted as a single file named *report.pdf*. They can also be submitted in this ipynb notebook, but equations wherever required, should be formatted using LaTeX math-mode.
* All discussion regarding practical problems, along with solutions and plots should be specified here itself. We will not generate the solutions/plots again by running your code.
* Your name, personal number and email address should be specified above and also in your file *report.pdf*.
* All datasets can be downloaded from the course website.
* All tables and other additional information should be included.

# Theoretical problems

## [Naive Bayes Classifier, 6 points]

A psychologist does a small survey on ''happiness''. Each respondent provides a vector with entries 1 or 0 corresponding to if they answered “yes” or “no” to a question respectively. The question vector has attributes 
$$
x = (\mbox{rich, married, healthy}) \tag{1}
$$

Thus a response $(1, 0, 1)$ would indicate that the respondent was
''rich'', ''unmarried'' and ''healthy''. In addition, each respondent
gives a value $c = 1$ if they are content wih their life and $c = 0$
if they’re not. The following responses were obtained.

$$
c = 1: (1, 1, 1),(0, 0, 1),(1, 1, 0),(1, 0, 1) \\
c = 0: (0, 0, 0),(1, 0, 0),(0, 0, 1),(0, 1, 0)
$$

1. Using naive Bayes, what is the probability that a person is ''not rich'', ''married'' and ''healthy'' is ''content''?

2. What is the probability that a person who is ''not rich'' and ''married'' is content ? (i.e. we do not know if they are ''healthy'')

## [Extending Naive Bayes, 4 points]

Consider now, the following vector of attributes:

* $x_1 = 1$ if customer is younger than 20 and 0 otherwise.
* $x_2 = 1$ if customer is between 20 and 30 in age, and 0 otherwise.
* $x_3 = 1$ if customer is older than 30 and 0 otherwise
* $x_4 = 1$ if customer walks to work and 0 otherwise.

Each vector of attributes has a label ''rich'' or ''poor''. Point out potential difficulties with your approach above to training using naive Bayes. Suggest and describe how to extend your naive Bayes method to this dataset.


## Solutions

See report

# Practical problems

## [Bayes classifier, 5 points]

Dowload the dataset **"dataset2.txt"**. You can use the following code for example:
```python
from numpy import genfromtxt
data = genfromtxt('dataset2.txt', delimiter=',')
labels = data[:,-1]
```
The dataset contains $3$-dimensional data, $X$, generated from $2$ classes with labels, $y$ either $+1$ or $-1$.  Each row of $X$ and $y$ contain one observation and one label respectively.  There are $1000$ instances of each class. 

a. Assume that the class conditional density is spherical Gaussian, and both classes have equal prior. Write the expression for the Bayes (<span style="color:red"> not **naive Bayes**</span>) classifier i.e. derive
$$
P(y_{new} = -1 | x_{new} , X, y ) \\
P(y_{new} = +1 | x_{new} , X, y ) ~.
$$

It is useful to note that the dependence on training data $X, y$ for class $1$ can be expressed as: 

$$ 
P( x_{new} | y_{new} = 1, X, y) = P(x_{new} |
\hat{\mu}_{1}, \hat{\sigma}^{2}_{1})
$$

where $\hat{\mu}_{1} \in \mathbb{R}^3$ and $\hat{\sigma}^{2}_{1}\in \mathbb{R}$ are MLE estimates for mean (3-dimensional) and variance based on training data with label $+1$ (and similarly for class 2 with label $-1$). 

b. Implement a function **sph_bayes()** which computes the probability of a new test point *Xtest* coming from class $1$ ($P1$) and class $2$ ($P2$). Finally, assign a label *Ytest* to the test point based on the probabilities $P1$ and $P2$.

```python
def sph_bayes(Xtest, ...): # other parameters needed.

    return [P1, P2, Ytest]
```
c. Write a function **new_classifier()**

```python
def new_classifier(Xtest, mu1, mu2)
    
    return [Ytest]
```
which implements the following classifier,
$$
f(x) = \mbox{sign}\left(\frac{(\mu_1 - \mu_2)^\top (x - b) }{\|\mu_1 -  \mu_2\|_2} \right)
$$
with $b = \frac{1}{2}(\mu_1 + \mu_2)$.

d. Report 5-fold cross validation error for both classifiers.


## Solution

a. See report

b-d. See below

In [22]:
import numpy as np
from numpy.linalg import inv, det, norm
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from collections import namedtuple


def sph_bayes(Xtest, mu1, mu2, var1, var2): # other parameters needed.
    dist1 = multivariate_normal(mean=mu1, cov=var1)
    dist2 = multivariate_normal(mean=mu2, cov=var2)
    
    lik1 = dist1.pdf(Xtest)
    lik2 = dist2.pdf(Xtest)
    
    marg_lik = lik1+lik2
    
    P1 = (lik1 / marg_lik)
    P2 = (lik2 / marg_lik)
    
    Ytest = (P1 > P2).astype(int) * 2.0 - 1.0

    return [P1, P2, Ytest]


def new_classifier(Xtest, mu1, mu2):
    Ytest = np.empty(Xtest.shape[0])
    b = (mu1 + mu2) / 2.0
    mu_diff = mu1 - mu2
    
    for i in range(Ytest.shape[0]):
        b_diff = Xtest[i,:] - b
        mu_dist = np.sqrt(np.sum(np.square(mu_diff)))
        Ytest[i] = np.dot(mu_diff, b_diff) / mu_dist
    
    return np.sign(Ytest)


def get_gauss_params(data):
    mu = np.mean(data, axis=0)
    var = np.var(data)
    return mu, var
    
    
CVResult = namedtuple('CVResult', ('err_counts', 'test_samples_counts', 'err_ratios'))

def cross_validate(data, labels, k, classifier):
    data1 = data[labels > 0,:]
    data2 = data[labels < 0,:]
    
    assert data1.shape[0] + data2.shape[0] == data.shape[0], "bad labels"
    assert data1.shape[0] > 0, "no sample for class 1"
    assert data2.shape[0] > 0, "no sample for class 2"
    assert np.unique(labels).shape[0] == 2, "more than two distinct labels"
    
    prior_mu1, prior_var1 = get_gauss_params(data1)
    prior_mu2, prior_var2 = get_gauss_params(data2)
    
    kf = KFold(n_splits=k)
    err_counts = []
    test_samples_counts = []
    err_ratios = []

    for train_index, test_index in kf.split(data):
        X_train, X_test = data[train_index], data[test_index]
        labels_train, labels_test = labels[train_index], labels[test_index]
        
        Y_train = classifier(X_train, prior_mu1, prior_mu2, prior_var1, prior_var2)
        post_mu1, post_var1 = get_gauss_params(X_train[Y_train > 0.0,:])
        post_mu2, post_var2 = get_gauss_params(X_train[Y_train < 0.0,:])
        Y_test = classifier(X_test, post_mu1, post_mu2, post_var1, post_var2)

        err_count = np.sum(~np.isclose(Y_test, labels_test))
        err_counts.append(err_count)
        
        n = X_test.shape[0]
        test_samples_counts.append(n)
    
    err_counts = np.array(err_counts)
    test_samples_counts = np.array(test_samples_counts)
        
    return CVResult(
        err_counts=err_counts,
        test_samples_counts=test_samples_counts,
        err_ratios=np.divide(err_counts, test_samples_counts),
    )


In [23]:
data = np.genfromtxt('dataset2.txt', delimiter=',')
labels = data[:,-1]
data = data[:,:3]

K = 5

res_sb = cross_validate(
    data=data,
    labels=labels,
    k=K,
    classifier=lambda X, mu1, mu2, var1, var2: sph_bayes(X, mu1, mu2, var1, var2)[2]
)

res_nc = cross_validate(
    data=data,
    labels=labels,
    k=K,
    classifier=lambda X, mu1, mu2, _var1, _var2: new_classifier(X, mu1, mu2)
)

print(f"Classification error rates per fold [{K}-fold]")
print("Spherical Bayes:  ", res_sb.err_ratios)
print('"New classifier": ', res_sb.err_ratios)
    

Classification error rates per fold [5-fold]
Spherical Bayes:   [ 0.  0.  0.  0.  0.]
"New classifier":  [ 0.  0.  0.  0.  0.]



## [DIGITS dataset classifer, 5 points]

Load the DIGITS dataset:
```python
from sklearn import datasets
digits = datasets.load_digits()
```
This dataset contains $1797$ samples of ten handwritten digit classes. You can further query and visualize the dataset using the various attributes of the returned dictionary:
```python
data = digits.data
print(data.shape)
target_names = digits.target_names
print (target_names)
import matplotlib.pyplot as plt
y = digits.target
plt.matshow(digits.images[0])
plt.show()
```

a. Use **new_classifier()** designed previously to do binary classification between classes representing digits "*5*" and "*8*".

b. Investigate an alternative feature function as described below:

1. Scale each pixel value to range $[0, 1] $ from original gray-scale ($0-255$). 
2. Compute variance of each row and column of the image. This will give you a new feature vector of size $16$ i.e. 

$$ 
x' = \left[ \; Var(row_1)  , Var(row_2), \ldots , Var(row_{8}), Var(col_1), \ldots, Var(col_{8}) \;\right]^T
$$

c. Report $5$-fold cross validation results for parts $(a)$ and
$(b)$ in a single table. What can you say about the results?

In [24]:
from sklearn import datasets
import matplotlib.pyplot as plt
from math import sqrt

digits = datasets.load_digits()
data = digits.data

Y = digits.target
subset = np.array([y in [5,8] for y in Y])
X = digits.data[subset, :]
labels = np.array([+1 if y == 5 else -1 for y in Y[subset]])

K = 5

res_a = cross_validate(
    data=X,
    labels=labels,
    k=K,
    classifier=lambda X, mu1, mu2, _var1, _var2: new_classifier(X, mu1, mu2)
)

def extract_features(X):
    X_features = []
    for i in range(X.shape[0]):
        x = X[i,:].reshape((8, 8))
        var_cols = np.var(x, axis=0)
        var_rows = np.var(x, axis=1)
        features = np.concatenate([var_rows, var_cols])
        
        X_features.append(features)
    
    return np.array(X_features)

assert np.min(X) >= 0.0
X_scaled = X / np.max(X)
X_prime = extract_features(X_scaled)

res_b = cross_validate(
    data=X_prime,
    labels=labels,
    k=K,
    classifier=lambda X, mu1, mu2, _var1, _var2: new_classifier(X, mu1, mu2)
)

print(f"Classification error rates per fold [{K}-fold]")
print("From pixels:            ", res_a.err_ratios)
print("From extracted features:", res_b.err_ratios)
    


Classification error rates per fold [5-fold]
From pixels:             [ 0.01388889  0.01408451  0.          0.05633803  0.02816901]
From extracted features: [ 0.25        0.15492958  0.12676056  0.12676056  0.16901408]


As we can see with the error rates above, this classifier performs better using the pixels directly than using the variance of each rows and columns, even though the dimensionality is higher in this case.